In [2]:
import pandas as pd
import numpy as np
import csv

In [2]:
df_paths = ['nss_annual/nss{0}.tsv'.format(i) for i in ['09'] + range(10,16)]
df_list = []
for p in df_paths:
    this_df = pd.read_csv(p, sep='\t', error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE)
    df_list.append(this_df)
    # printing out columns to quickly verify that there weren't any year-to-year column changes
    print p
    print '\t{0}'.format(list(this_df.columns))
df_all = pd.concat(df_list, ignore_index=True)
df_all.to_csv('nss_processed/raw_complete.tsv', sep='\t', index=False)

nss_annual/nss09.tsv
	['CPSC Case #', 'trmt_date', 'psu', 'weight', 'stratum', 'age', 'sex', 'race', 'race_other', 'diag', 'diag_other', 'body_part', 'disposition', 'location', 'fmv', 'prod1', 'prod2', 'narr1', 'narr2']


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


nss_annual/nss10.tsv
	['CPSC Case #', 'trmt_date', 'psu', 'weight', 'stratum', 'age', 'sex', 'race', 'race_other', 'diag', 'diag_other', 'body_part', 'disposition', 'location', 'fmv', 'prod1', 'prod2', 'narr1', 'narr2']
nss_annual/nss11.tsv
	['CPSC Case #', 'trmt_date', 'psu', 'weight', 'stratum', 'age', 'sex', 'race', 'race_other', 'diag', 'diag_other', 'body_part', 'disposition', 'location', 'fmv', 'prod1', 'prod2', 'narr1', 'narr2']
nss_annual/nss12.tsv
	['CPSC Case #', 'trmt_date', 'psu', 'weight', 'stratum', 'age', 'sex', 'race', 'race_other', 'diag', 'diag_other', 'body_part', 'disposition', 'location', 'fmv', 'prod1', 'prod2', 'narr1', 'narr2']
nss_annual/nss13.tsv
	['CPSC Case #', 'trmt_date', 'psu', 'weight', 'stratum', 'age', 'sex', 'race', 'race_other', 'diag', 'diag_other', 'body_part', 'disposition', 'location', 'fmv', 'prod1', 'prod2', 'narr1', 'narr2']
nss_annual/nss14.tsv
	['CPSC Case #', 'trmt_date', 'psu', 'weight', 'stratum', 'age', 'sex', 'race', 'race_other', 'diag

In [3]:
# I start my ipynb sessions from here
full_data = 'nss_processed/raw_complete.tsv'
df_all = pd.read_csv(full_data, sep='\t', error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
def consolidate_narr_cols():
    """
    consolidate narrative columns - mutates df_all in place
    """
    ignore_nans = lambda s: s if isinstance(s, str) else ''
    narr_ptr = df_all.loc[:,['narr1','narr2']] 
    df_all['narr'] = narr_ptr['narr1'].apply(ignore_nans) + narr_ptr['narr2'].apply(ignore_nans)
consolidate_narr_cols()
df_all.head()

,CPSC Case #,trmt_date,psu,weight,stratum,age,sex,race,race_other,diag,diag_other,body_part,disposition,location,fmv,prod1,prod2,narr1,narr2,narr
0,90101432,01/01/2009,61,15.3491,V,5,1,3,HISPANIC,64,NaN,89,1,1,0,1807,NaN,5 YOM ROLLING ON FLOOR DOING A SOMERSAULT AND ...,IN,5 YOM ROLLING ON FLOOR DOING A SOMERSAULT AND ...
1,90101434,01/01/2009,61,15.3491,V,51,1,1,NaN,53,NaN,77,1,1,0,899,NaN,"51 YOM C/O PAIN AND IRRITATION TO RIGHT EYE, H...","HOME AND POSSIBLY THE CAUSE, FOUND TO HAVE MET...","51 YOM C/O PAIN AND IRRITATION TO RIGHT EYE, H..."
2,90101435,01/01/2009,61,15.3491,V,2,2,1,NaN,59,NaN,76,1,1,0,4057,NaN,2 YOF WAS RUNNING THROUGH HOUSE AND FELL INTO ...,G A LACERATION TO FACE NEAR INSIDE CORNER OF R...,2 YOF WAS RUNNING THROUGH HOUSE AND FELL INTO ...
3,90101436,01/01/2009,61,15.3491,V,20,1,1,NaN,53,NaN,93,1,1,0,1884,NaN,20 YOM PUNCHED AND KICKED A WALL D/T DRINKING ...,AINED CONTUSIONS AND ABRASIONS TO RIGHT MIDDLE...,20 YOM PUNCHED AND KICKED A WALL D/T DRINKING ...
4,90101437,01/01/2009,61,15.3491,V,20,1,1,NaN,57,NaN,34,1,9,0,3283,NaN,20 YOM FELL SNOW SKIING SUSTAINING A FRACTURED...,NaN,20 YOM FELL SNOW SKIING SUSTAINING A FRACTURED...


In [5]:
# extract month information from dates
def extract_month(dt_str , fmt='%m/%d/%Y'):
    """
    extract month information from dates
    :param date_str:
    :param fmt:
    :return:
    """
    from datetime import datetime as dt # only imports once, no matter how often it's called
    return int(dt.strptime(dt_str, fmt).strftime('%m'))
df_all['month'] = df_all['trmt_date'].apply(extract_month)
df_all.head()

,CPSC Case #,trmt_date,psu,weight,stratum,age,sex,race,race_other,diag,...,body_part,disposition,location,fmv,prod1,prod2,narr1,narr2,narr,month
0,90101432,01/01/2009,61,15.3491,V,5,1,3,HISPANIC,64,...,89,1,1,0,1807,NaN,5 YOM ROLLING ON FLOOR DOING A SOMERSAULT AND ...,IN,5 YOM ROLLING ON FLOOR DOING A SOMERSAULT AND ...,1
1,90101434,01/01/2009,61,15.3491,V,51,1,1,NaN,53,...,77,1,1,0,899,NaN,"51 YOM C/O PAIN AND IRRITATION TO RIGHT EYE, H...","HOME AND POSSIBLY THE CAUSE, FOUND TO HAVE MET...","51 YOM C/O PAIN AND IRRITATION TO RIGHT EYE, H...",1
2,90101435,01/01/2009,61,15.3491,V,2,2,1,NaN,59,...,76,1,1,0,4057,NaN,2 YOF WAS RUNNING THROUGH HOUSE AND FELL INTO ...,G A LACERATION TO FACE NEAR INSIDE CORNER OF R...,2 YOF WAS RUNNING THROUGH HOUSE AND FELL INTO ...,1
3,90101436,01/01/2009,61,15.3491,V,20,1,1,NaN,53,...,93,1,1,0,1884,NaN,20 YOM PUNCHED AND KICKED A WALL D/T DRINKING ...,AINED CONTUSIONS AND ABRASIONS TO RIGHT MIDDLE...,20 YOM PUNCHED AND KICKED A WALL D/T DRINKING ...,1
4,90101437,01/01/2009,61,15.3491,V,20,1,1,NaN,57,...,34,1,9,0,3283,NaN,20 YOM FELL SNOW SKIING SUSTAINING A FRACTURED...,NaN,20 YOM FELL SNOW SKIING SUSTAINING A FRACTURED...,1


In [6]:
# NEISS encodes age in months as 2XX where XX is the number of months
def standardize_age():
    """
    NEISS encodes age in months as 2XX where XX is the number of months. 
    :return: standardized float vector representing age in years
    """
    standardize_age = lambda age: (age - 200)/12.0 if age > 200 else float(age)
    df_all['age'] = df_all['age'].apply(standardize_age)
standardize_age()
print df_all.head()

   CPSC Case #   trmt_date  psu   weight stratum  age  sex  race race_other  \
0     90101432  01/01/2009   61  15.3491       V    5    1     3   HISPANIC   
1     90101434  01/01/2009   61  15.3491       V   51    1     1        NaN   
2     90101435  01/01/2009   61  15.3491       V    2    2     1        NaN   
3     90101436  01/01/2009   61  15.3491       V   20    1     1        NaN   
4     90101437  01/01/2009   61  15.3491       V   20    1     1        NaN   

   diag  ...  body_part  disposition  location  fmv  prod1  prod2  \
0    64  ...         89            1         1    0   1807    NaN   
1    53  ...         77            1         1    0    899    NaN   
2    59  ...         76            1         1    0   4057    NaN   
3    53  ...         93            1         1    0   1884    NaN   
4    57  ...         34            1         9    0   3283    NaN   

                                               narr1  \
0  5 YOM ROLLING ON FLOOR DOING A SOMERSAULT AND ...  

In [7]:
# convert categorical/nominal variables into binary indicators
def nominal_to_dummies(nom_features):
    """
    convert categorical/nominal variables into binary indicators
    :param nom_features:
    :return:
    """
    safe_tostr = lambda val: str(val) if pd.notnull(val) else val
    for nom in nom_features: # remove :1 to include all variables
        df_all.loc[:,nom] = df_all[nom].apply(safe_tostr)
    dummy_df = pd.get_dummies(df_all.loc[:,nom_features])
    return dummy_df
### do the following online
nom_feats = ['sex', 'race', 'stratum', 'diag', 'body_part', 'location']
processed_df = nominal_to_dummies(nom_feats)
processed_df.head()

,sex_0,sex_1,sex_2,race_0,race_1,race_2,race_3,race_4,race_5,race_6,...,body_part_94,location_0,location_1,location_2,location_4,location_5,location_6,location_7,location_8,location_9
0,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
# combine extracted binary features with transformed features we want to keep
to_add = ['month', 'age','disposition'] # TODO: add narr after performing feature extraction
for feat in to_add:
    processed_df[feat] = df_all[feat]
processed_df.head()

,sex_0,sex_1,sex_2,race_0,race_1,race_2,race_3,race_4,race_5,race_6,...,location_2,location_4,location_5,location_6,location_7,location_8,location_9,month,age,disposition
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,5,1
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,51,1
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,1
3,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,20,1
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,20,1


In [235]:
# assess conditional dependence of outcomes on products given diagnosis and body part 
### build out with one combination of diagnosis/body part, then iterate through all of them reporting N and 
### chi-squared statistic. try with just product codes only first, on unlikely chance there is no unconditioned relationship.
this_diag = 59
this_body_part = 75

# delete after use:
full_data = 'nss_processed/raw_complete.tsv'
df_all = pd.read_csv(full_data, sep='\t', error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE)
df_all = df_all.loc[(df_all.diag == this_diag) & (df_all.body_part == this_body_part)]
print df_all.head()

     CPSC Case #   trmt_date  psu   weight stratum  age  sex  race race_other  \
15      90101456  01/01/2009   61  15.3491       V    5    1     3    UNKNOWN   
65      90101831  01/02/2009   77  15.3491       V   12    1     3   HISPANIC   
66      90101832  01/02/2009   77  15.3491       V    2    1     0        NaN   
73      90101863  01/03/2009   77  15.3491       V    7    2     2        NaN   
113     90102553  01/02/2009   72  70.8387       S   52    2     1        NaN   

     diag diag_other  body_part  disposition  location  fmv  prod1  prod2  \
15     59        NaN         75            1         1    0   4057    NaN   
65     59        NaN         75            1         1    0   4057    NaN   
66     59        NaN         75            1         1    0   4057    NaN   
73     59        NaN         75            1         1    0   1842    NaN   
113    59        NaN         75            1         1    0   1842    NaN   

                                                 n

In [236]:
outcome_df = pd.DataFrame(df_all.disposition.value_counts()).transpose()
prod_df = pd.DataFrame(df_all.prod1.value_counts()).transpose()
total = float(sum(outcome_df.ix[0]))
print outcome_df
print prod_df
print total

       1     4    6    5    2   8  9
0  63085  2311  582  293  118  14  1
   1807  4076  4057  1842  1884  4074  4056  1893  5040  604   ...   1936  \
0  6135  5392  4591  4084  3735  2812  2522  1803  1746  1540  ...      1   

   529   1553  1462  979   3218  1619  981   373   384   
0     1     1     1     1     1     1     1     1     1  

[1 rows x 566 columns]
66404.0


In [237]:
# calcluate create dataframe of expected values under independence
# and aggregate observed values for comparison
exp_df = pd.DataFrame(index=prod_df.columns, columns=outcome_df.columns)
obs_df = pd.DataFrame(index=prod_df.columns, columns=outcome_df.columns)
for dis in outcome_df.columns:
    for prod in prod_df.columns:
        obs_df.loc[prod,dis] = len(df_all.loc[(df_all.prod1 == prod) & (df_all.disposition == dis)])
        exp_df.loc[prod,dis] = (prod_df[prod][0]/total)*outcome_df[dis][0]
print exp_df
print obs_df

              1           4            6            5            2  \
1807   5828.361     213.511     53.77041     27.06998      10.9019   
4076   5122.497     187.653     47.25836     23.79158     9.581591   
4057   4361.533    159.7765     40.23797     20.25726     8.158213   
1842   3879.874    142.1319     35.79435     18.02018     7.257274   
1884   3548.317    129.9859     32.73553     16.48026       6.6371   
4074   2671.451    97.86356     24.64586     12.40763     4.996928   
4056   2395.946    87.77095     22.10415     11.12803     4.481597   
1893   1712.883    62.74822     15.80245      7.95553     3.203933   
5040   1658.732     60.7645     15.30287     7.704024     3.102644   
604    1463.028    53.59527     13.49738     6.795073     2.736582   
611    1460.178    53.49086     13.47109     6.781835     2.731251   
679    1153.322    42.24977     10.64014     5.356635      2.15728   
1205   908.2173    33.27083     8.378893      4.21824     1.698813   
1878   637.4621    2

In [241]:
# conduct chi-squared test
import scipy.stats as stats
chisq = (((obs_df - exp_df)**2)/exp_df).sum().sum()
print chisq
degrees_free = (len(obs_df.columns) - 1)*(len(obs_df.index) - 1)
p = 1.0 - stats.chi2.cdf(x=chisq, df=degrees_free)
print p

5080.53640839
0.0


In [9]:
# looks like we should be using product codes after all
# distance-based clustering isn't appropriate for all the binary variables here
# certain nlp methods could be helpful, but would take too long to implement
# let's capture a bayesian likelihood so that we're utilizing some product information
import pickle
outcome_df = pd.DataFrame(df_all.disposition.value_counts()).transpose()
etl_rec_dir = 'neiss/etl_utils/resources/'
pickle.dump(outcome_df, open(etl_rec_dir + 'total_outcome_freqs.pkl','wb+'))
print outcome_df.head() ### delete this
for target in sorted(df_all.disposition.unique()):
    new_col = 'prod_targ_{0}'.format(target)
    print new_col ### delete
    disp_df = df_all[df_all['disposition'] == target]['prod1'].value_counts()
    # create dictionary of products and their observed frequencies with these outcomes
    disp_dict = {prod:disp_df[prod] for prod in disp_df.index}
    pickle.dump(disp_dict, open(etl_rec_dir + '{0}.pkl'.format(new_col),'wb+'))
    total_target = float(outcome_df[target][0])
    processed_df[new_col] = [disp_dict[prod]/total_target if prod in disp_dict 
                             else 1/total_target
                             for prod in df_all['prod1'].values]    

         1       4      6      2      5     8    9
0  2453099  173623  28870  18855  16361  1138  140
prod_targ_1
prod_targ_2
prod_targ_4
prod_targ_5
prod_targ_6
prod_targ_8
prod_targ_9


In [23]:
# reorder columns alphabetically for consistency of model inputs
ordered_cols = sorted(list(processed_df.columns))
print ordered_cols
processed_df = processed_df[ordered_cols]
processed_df.head()

['age', 'body_part_0', 'body_part_30', 'body_part_31', 'body_part_32', 'body_part_33', 'body_part_34', 'body_part_35', 'body_part_36', 'body_part_37', 'body_part_38', 'body_part_75', 'body_part_76', 'body_part_77', 'body_part_79', 'body_part_80', 'body_part_81', 'body_part_82', 'body_part_83', 'body_part_84', 'body_part_85', 'body_part_87', 'body_part_88', 'body_part_89', 'body_part_92', 'body_part_93', 'body_part_94', 'diag_41', 'diag_42', 'diag_46', 'diag_47', 'diag_48', 'diag_49', 'diag_50', 'diag_51', 'diag_52', 'diag_53', 'diag_54', 'diag_55', 'diag_56', 'diag_57', 'diag_58', 'diag_59', 'diag_60', 'diag_61', 'diag_62', 'diag_63', 'diag_64', 'diag_65', 'diag_66', 'diag_67', 'diag_68', 'diag_69', 'diag_71', 'diag_72', 'diag_73', 'diag_74', 'disposition', 'location_0', 'location_1', 'location_2', 'location_4', 'location_5', 'location_6', 'location_7', 'location_8', 'location_9', 'month', 'prod_targ_1', 'prod_targ_2', 'prod_targ_4', 'prod_targ_5', 'prod_targ_6', 'prod_targ_8', 'prod_t

,age,body_part_0,body_part_30,body_part_31,body_part_32,body_part_33,body_part_34,body_part_35,body_part_36,body_part_37,...,race_5,race_6,sex_0,sex_1,sex_2,stratum_C,stratum_L,stratum_M,stratum_S,stratum_V
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,51,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,20,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,20,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [24]:
processed_df.to_csv('nss_processed/processed_df_sorted.tsv', sep='\t', index=False)

In [25]:
# start ipynb from here in modeling stage
import pandas as pd
import numpy as np

feat_df = pd.read_csv('nss_processed/processed_df_sorted.tsv', sep='\t')

In [26]:
# sampling with replacement based on survey weights - makes sample representative of population
print len(feat_df)
feat_df = feat_df.sample(n=int(len(feat_df)*1.0), weights=df_all.weight, replace=True, random_state=15)

2692086


In [34]:
print 'disposition' in feat_df.columns
target = feat_df.disposition.values
feat_df.drop('disposition', axis=1, inplace=True)
print 'disposition' in feat_df.columns

False


AttributeError: 'DataFrame' object has no attribute 'disposition'

In [28]:
# candidates to try:
# Naive bayes - multinomialNB w/ weight vector as statistical weight - need to reduce dimensions
# Random Forest
### KNN (and RF) w/ dimensionality reduction techniques to consider: PCA, LDA
### apply regularization
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feat_df, target, test_size=0.2, random_state=15)

In [29]:
# fit random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10, random_state=15)
rf_fit = rf.fit(X_train, y_train)

In [30]:
# fit random forest
y_hat = rf_fit.predict(X_test)

In [31]:
# evaluate random forest

# the overall accuracy alone can be misleading for these unbalanced data
print 'overall classificaiton accuracy: {0:.3f}%'.format(np.mean(y_hat == y_test)*100)
for i in list(set(y_test)):
    conditional_acc = np.mean([hat == test for hat,test in zip(y_hat,y_test) if test == i])
    print 'accuracy for disposition = {0}: {1:.3f}%'.format(i,conditional_acc*100)
    print '\ttotal disp={0} observations: {1}'.format(i, len([t for t in y_test if t ==i]))
    prop = len([t for t in y_test if t ==i])/float(len(y_test))*100
    print '\t percentage disp={0}: {1:.3f}%'.format(i,prop)

overall classificaiton accuracy: 96.590%
accuracy for disposition = 1: 99.171%
	total disp=1 observations: 492831
	 percentage disp=1: 91.533%
accuracy for disposition = 2: 69.668%
	total disp=2 observations: 5753
	 percentage disp=2: 1.069%
accuracy for disposition = 4: 71.471%
	total disp=4 observations: 31540
	 percentage disp=4: 5.858%
accuracy for disposition = 5: 54.757%
	total disp=5 observations: 2365
	 percentage disp=5: 0.439%
accuracy for disposition = 6: 58.420%
	total disp=6 observations: 5659
	 percentage disp=6: 1.051%
accuracy for disposition = 8: 59.244%
	total disp=8 observations: 238
	 percentage disp=8: 0.044%
accuracy for disposition = 9: 59.375%
	total disp=9 observations: 32
	 percentage disp=9: 0.006%


In [33]:
# serialize model
from sklearn.externals import joblib
from datetime import datetime as dt
description = 'rf_weighted_resampling'
p_path = 'models/{datestr}_{desc}.pkl'.format(datestr=dt.strftime(dt.now(),'%Y%m%d-%H-%M-%S'), desc=description)
joblib.dump(rf_fit, p_path)

['models/20161205-01-43-30_rf_weighted_resampling.pkl']

In [36]:
from sklearn.externals import joblib
load_test = joblib.load('neiss/' + p_path)
y_hat = rf_fit.predict(X_test)

In [44]:
X_test.ix[2238263]

,age,body_part_0,body_part_30,body_part_31,body_part_32,body_part_33,body_part_34,body_part_35,body_part_36,body_part_37,...,race_5,race_6,sex_0,sex_1,sex_2,stratum_C,stratum_L,stratum_M,stratum_S,stratum_V
2238263,48,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2238263,48,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0


In [45]:
print p_path

models/20161205-01-43-30_rf_weighted_resampling.pkl


In [47]:
sorted({1,2,3})

[1, 2, 3]